In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# The forward and backward passes

In [2]:
#export
from exp.nb_01 import *

In [3]:
def x_to_tensor(listed):
    return tensor(list(map(array, listed)), dtype=torch.float32).reshape(len(listed), -1)

def y_to_tensor(listed):
    return tensor(array(listed))

In [6]:
def get_data():
    mnist = DataBlock(blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
                     get_items = get_image_files,
                     splitter = GrandparentSplitter('training', 'testing'),
                     get_y = parent_label)
    dls = mnist.dataloaders(untar_data(URLs.MNIST))
    
    x_train, y_train, x_valid, y_valid = [], [], [], []
    
    for elem in dls.train_ds:
        x_train.append(elem[0])
        y_train.append(elem[1])
    for elem in dls.valid_ds:
        x_valid.append(elem[0])
        y_valid.append(elem[1])
        
    return x_to_tensor(x_train), y_to_tensor(y_train), x_to_tensor(x_valid), y_to_tensor(y_valid)
    
def normalize(x, m, s): return (x-m)/s

In [24]:
%time x_train, y_train, x_valid, y_valid = get_data()

CPU times: user 42 s, sys: 1.18 s, total: 43.2 s
Wall time: 42.8 s


In [25]:
x_train, x_valid = x_train / 255.0, x_valid / 255.0

In [26]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1307), tensor(0.3081))

In [27]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [28]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(-5.9790e-09), tensor(1.0000))

In [29]:
#export
def test_near_zero(a, tol=1e-3): assert a.abs() < tol, f'Not Near zero: {a}'

In [31]:
test_near_zero(train_mean)
test_near_zero(1 - train_std)

In [32]:
n, m = x_train.shape
c = y_train.max() + 1
n, m, c

(60000, 784, tensor(10))

# Foundations version

## Basic architecture

In [33]:
# num hidden
nh = 50

In [34]:
# standard xavier init
w1 = torch.randn(m,nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1) / math.sqrt(nh)
b2 = torch.zeros(1)

In [36]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1 / math.sqrt(m))

In [41]:
w1.std(), 1 / math.sqrt(m)

(tensor(0.0357), 0.03571428571428571)

In [42]:
x_valid.mean(), x_valid.std() # ~ 0, 1

(tensor(0.0060), tensor(1.0077))

In [43]:
def lin(x, w, b): return x@w + b

In [44]:
t = lin(x_valid, w1, b1)

In [45]:
t.mean(), t.std() # ~ 0, 1

(tensor(0.1280), tensor(0.9558))

In [46]:
def relu(x): return x.clamp_min(0.)

In [47]:
t = relu(lin(x_valid, w1, b1))

In [48]:
t.mean(), t.std() # ~ pretty bad thanks to the relu

(tensor(0.4466), tensor(0.6024))

In [53]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh) * math.sqrt(2/m)

In [54]:
w1.mean(), w1.std()

(tensor(-2.8252e-05), tensor(0.0507))

In [55]:
t = relu(lin(x_valid, w1, b1))

In [56]:
t.mean(), t.std()

(tensor(0.4951), tensor(0.7637))

In [57]:
# export
from torch.nn import init

In [58]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') # fan_out because in pytorch the tensors are batchsize first!
t = relu(lin(x_valid, w1, b1))

In [61]:
w1.mean(), w1.std()

(tensor(-9.2173e-06), tensor(0.0504))

In [62]:
t.mean(), t.std()

(tensor(0.5189), tensor(0.7690))

In [63]:
w1.shape

torch.Size([784, 50])

In [60]:
init.kaiming_normal_??

Signature: init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
Source:   
def kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std}^2)` where

    .. math::
        \text{std} = \frac{\text{gain}}{\sqrt{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``

In [67]:
import torch.nn

In [69]:
m, nh

(784, 50)

In [68]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [70]:
# what if...
def relu(x): return x.clamp_min(0.) - 0.5

In [71]:
w1 = torch.randn(m,nh) * math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.0384), tensor(0.7900))

In [72]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [73]:
%timeit -n 10 _ = model(x_valid)

6.92 ms ± 427 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [74]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

# Loss function: MSE

In [75]:
model(x_valid).shape

torch.Size([10000, 1])

In [76]:
#export
def mse(output, target): return (output.squeeze(-1) - target).pow(2).mean()

In [77]:
y_train, y_valid = y_train.float(), y_valid.float()

In [78]:
preds = model(x_train)

In [79]:
preds.shape

torch.Size([60000, 1])

In [80]:
mse(preds, y_train)

tensor(26.8211)

# Gradients and backward pass